In [21]:
# Python 3.x Utility Functions

import re
import numpy as np
import math
import random
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque, abc, OrderedDict
from functools   import lru_cache
from itertools   import (permutations, combinations, chain, cycle, product, islice, 
                         takewhile, zip_longest, count as count_from)
from heapq       import heappop, heappush

identity = lambda x: x
letters  = 'abcdefghijklmnopqrstuvwxyz'

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

################ Functions for Input, Parsing

def Input(day):
    "Open this day's input file."
    return open('day{0:02d}.txt'.format(day))
    
def array(lines):
    "Parse an iterable of str lines into a 2-D array. If `lines` is a str, do splitlines."
    if isinstance(lines, str): lines = lines.splitlines()
    return mapt(vector, lines)

def vector(line):
    "Parse a str into a tuple of atoms (numbers or str tokens)."
    return mapt(atom, line.split())

def atom(token):
    "Parse a str token into a number, or leave it as a str."
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token

################ Functions on Iterables

def first(iterable, default=None): return next(iter(iterable), default)

def first_true(iterable, pred=None, default=None):
    """Returns the first true value in the iterable.
    If no true value is found, returns *default*
    If *pred* is not None, returns the first item
    for which pred(item) is true."""
    # first_true([a,b,c], default=x) --> a or b or c or x
    # first_true([a,b], fn, x) --> a if fn(a) else b if fn(b) else x
    return next(filter(pred, iterable), default)

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

def upto(iterable, maxval):
    "From a monotonically increasing iterable, generate all the values <= maxval."
    # Why <= maxval rather than < maxval? In part because that's how Ruby's upto does it.
    return takewhile(lambda x: x <= maxval, iterable)

def groupby(iterable, key=identity):
    "Return a dict of {key(item): [items...]} grouping all items in iterable by keys."
    groups = defaultdict(list)
    for item in iterable:
        groups[key(item)].append(item)
    return groups

def grouper(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks:
    grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"""
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

def overlapping(iterable, n):
    """Generate all (overlapping) n-element subsequences of iterable.
    overlapping('ABCDEFG', 3) --> ABC BCD CDE DEF EFG"""
    if isinstance(iterable, abc.Sequence):
        yield from (iterable[i:i+n] for i in range(len(iterable) + 1 - n))
    else:
        result = deque(maxlen=n)
        for x in iterable:
            result.append(x)
            if len(result) == n:
                yield tuple(result)
                
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    return overlapping(iterable, 2)

def sequence(iterable, type=tuple):
    "Coerce iterable to sequence: leave it alone if it is already a sequence, else make it of type."
    return iterable if isinstance(iterable, abc.Sequence) else type(iterable)

def join(iterable, sep=''):
    "Join the items in iterable, converting each to a string first."
    return sep.join(map(str, iterable))
                
def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c
            
def quantify(iterable, pred=bool):
    "Count how many times the predicate is true."
    return sum(map(pred, iterable))

def shuffled(iterable):
    "Create a new list out of iterable, and shuffle it."
    new = list(iterable)
    random.shuffle(new)
    return new
    
flatten = chain.from_iterable
            
class Set(frozenset):
    "A frozenset, but with a prettier printer."
    def __repr__(self): return '{' + join(sorted(self), ', ') + '}'
    
def canon(items, typ=None):
    "Canonicalize these order-independent items into a hashable canonical form."
    typ = typ or (cat if isinstance(items, str) else tuple)
    return typ(sorted(items))

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))
            
################ Math Functions
            
def transpose(matrix): return tuple(zip(*matrix))

def isqrt(n):
    "Integer square root (rounds down)."
    return int(n ** 0.5)

def ints(start, end):
    "The integers from start to end, inclusive: range(start, end+1)"
    return range(start, end + 1)

def floats(start, end, step=1.0):
    "Yields from start to end (inclusive), by increments of step."
    m = (1.0 if step >= 0 else -1.0)
    while start * m <= end * m:
        yield start
        start += step
        
def multiply(numbers):
    "Multiply all the numbers together."
    result = 1
    for n in numbers:
        result *= n
    return result

import operator as op

operations = {'>': op.gt, '>=': op.ge, '==': op.eq,
              '<': op.lt, '<=': op.le, '!=': op.ne,
              '+': op.add, '-': op.sub, '*': op.mul, 
              '/': op.truediv, '**': op.pow}

################ 2-D points implemented using (x, y) tuples

def X(point): x, y = point; return x
def Y(point): x, y = point; return y

origin = (0, 0)
UP, DOWN, LEFT, RIGHT = (0, 1), (0, -1), (-1, 0), (1, 0)

def neighbors4(point): 
    "The four neighboring squares."
    x, y = point
    return (          (x, y-1),
            (x-1, y),           (x+1, y), 
                      (x, y+1))

def neighbors8(point): 
    "The eight neighboring squares."
    x, y = point 
    return ((x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x-1, y+1), (x, y+1), (x+1, y+1))

def cityblock_distance(p, q=origin): 
    "Manhatten distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def distance(p, q=origin): 
    "Hypotenuse distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

################ Debugging 

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def grep(pattern, iterable):
    "Print lines from iterable that match pattern."
    for line in iterable:
        if re.search(pattern, line):
            print(line)

################ A* and Breadth-First Search (tracking states, not actions)

def always(value): return (lambda *args: value)

def Astar(start, moves_func, h_func, cost_func=always(1)):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    Path      = lambda s: ([] if (s is None) else Path(previous[s]) + [s])
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(s)
        for s2 in moves_func(s):
            g = path_cost[s] + cost_func(s, s2)
            if s2 not in path_cost or g < path_cost[s2]:
                heappush(frontier, (g + h_func(s2), s2))
                path_cost[s2] = g
                previous[s2] = s

def bfs(start, moves_func, goals):
    "Breadth-first search"
    goal_func = (goals if callable(goals) else lambda s: s in goals)
    return Astar(start, moves_func, lambda s: (0 if goal_func(s) else 1))

print("Done!")

Done!


In [18]:
def tests():
    # Functions for Input, Parsing
    assert array('''1 2 3
                    4 5 6''') == ((1, 2, 3), 
                                  (4, 5, 6))
    assert vector('testing 1 2 3.') == ('testing', 1, 2, 3.0)
    
    # Functions on Iterables
    assert first('abc') == first(['a', 'b', 'c']) == 'a'
    assert first_true([0, None, False, {}, 42, 43]) == 42
    assert nth('abc', 1) == nth(iter('abc'), 1) == 'b'
    assert cat(upto('abcdef', 'd')) == 'abcd'
    assert cat(['do', 'g']) == 'dog'
    assert groupby([-3, -2, -1, 1, 2], abs) == {1: [-1, 1], 2: [-2, 2], 3: [-3]}
    assert list(grouper(range(8), 3)) == [(0, 1, 2), (3, 4, 5), (6, 7, None)]
    assert list(overlapping((0, 1, 2, 3, 4), 3)) == [(0, 1, 2), (1, 2, 3), (2, 3, 4)]
    assert list(overlapping('abcdefg', 4)) == ['abcd', 'bcde', 'cdef', 'defg']  
    assert list(pairwise((0, 1, 2, 3, 4))) == [(0, 1), (1, 2), (2, 3), (3, 4)]
    assert sequence('seq') == 'seq'
    assert sequence((i**2 for i in range(5))) == (0, 1, 4, 9, 16)
    assert join(range(5)) == '01234'
    assert join(range(5), ', ') == '0, 1, 2, 3, 4'
    assert multiply([1, 2, 3, 4]) == 24
    assert transpose(((1, 2, 3), (4, 5, 6))) == ((1, 4), (2, 5), (3, 6))
    assert isqrt(9) == 3 == isqrt(10)
    assert ints(1, 100) == range(1, 101)
    assert identity('anything') == 'anything'
    assert set(powerset({1, 2, 3})) == {(), (1,), (1, 2), (1, 2, 3), (1, 3), (2,), (2, 3), (3,)}
    assert quantify(['testing', 1, 2, 3, int, len], callable) == 2 # int and len are callable
    assert quantify([0, False, None, '', [], (), {}, 42]) == 1  # Only 42 is truish
    assert set(shuffled('abc')) == set('abc')
    assert canon('abecedarian') == 'aaabcdeeinr'
    assert canon([9, 1, 4]) == canon({1, 4, 9}) == (1, 4, 9)
    assert mapt(math.sqrt, [1, 9, 4]) == (1, 3, 2)
    
    # Math
    assert transpose([(1, 2, 3), (4, 5, 6)]) == ((1, 4), (2, 5), (3, 6))
    assert isqrt(10) == isqrt(9) == 3
    assert ints(1, 5) == range(1, 6)
    assert list(floats(1, 5)) == [1., 2., 3., 4., 5.]
    assert multiply(ints(1, 10)) == math.factorial(10) == 3628800
    
    # 2-D points
    P = (3, 4)
    assert X(P) == 3 and Y(P) == 4
    assert cityblock_distance(P) == cityblock_distance(P, origin) == 7
    assert distance(P) == distance(P, origin) == 5
    
    # Search
    assert Astar((4, 4), neighbors8, distance) == [(4, 4), (3, 3), (2, 2), (1, 1), (0, 0)]
    assert bfs((4, 4), neighbors8, {origin}) == [(4, 4), (3, 3), (2, 2), (1, 1), (0, 0)]
    forty2 = always(42)
    assert forty2() == forty2('?') == forty2(4, 2) == 42

    return 'pass'

tests()

'pass'

In [8]:
## Advent Day 01

input_string = "61697637962276641366442297247367117738114719863473648131982449728688116728695866572989524473392982963976411147683588415878214189996163533584547175794158118148724298832798898333399786561459152644144669959887341481968319172987357989785791366732849932788343772112176614723858474959919713855398876956427631354172668133549845585632211935573662181331613137869866693259374322169811683635325321597242889358147123358117774914653787371368574784376721652181792371635288376729784967526824915192526744935187989571347746222113625577963476141923187534658445615596987614385911513939292257263723518774888174635963254624769684533531443745729344341973746469326838186248448483587477563285867499956446218775232374383433921835993136463383628861115573142854358943291148766299653633195582135934544964657663198387794442443531964615169655243652696782443394639169687847463721585527947839992182415393199964893658322757634675274422993237955354185194868638454891442893935694454324235968155913963282642649968153284626154111478389914316765783434365458352785868895582488312334931317935669453447478936938533669921165437373741448378477391812779971528975478298688754939216421429251727555596481943322266289527996672856387648674166997731342558986575258793261986817177487197512282162964167151259485744835854547513341322647732662443512251886771887651614177679229984271191292374755915457372775856178539965131319568278252326242615151412772254257847413799811417287481321745372879513766235745347872632946776538173667371228977212143996391617974367923439923774388523845589769341351167311398787797583543434725374343611724379399566197432154146881344528319826434554239373666962546271299717743591225567564655511353255197516515213963862383762258959957474789718564758843367325794589886852413314713698911855183778978722558742329429867239261464773646389484318446574375323674136638452173815176732385468675215264736786242866295648997365412637499692817747937982628518926381939279935993712418938567488289246779458432179335139731952167527521377546376518126276"
digits = mapt(int, input_string)
N = len(digits)

print(sum(digits[i] 
    for i in range(N) 
    if digits[i] == digits[i - 1]))

print(sum(digits[i] 
    for i in range(N) 
    if digits[i] == digits[i - N//2]))


1182
1152


In [9]:
## Advent Day 02

rows = array(Input(2).read())

print(sum(abs(max(row) - min(row)) for row in rows))

def evendiv(row): 
    return first(a // b for a in row for b in row if a > b and a // b == a / b)

print(sum(map(evendiv, rows)))

54426
333


In [10]:
## Advent Day 03

N = 277678

def spiral():
    "Yield the (x, y) coordinates of successive points in an infinite spiral."
    length = 1
    square = [0, 0]
    yield tuple(square)
    while True:
        yield from leg(square, length, RIGHT)
        yield from leg(square, length, UP)
        length += 1
        yield from leg(square, length, LEFT)
        yield from leg(square, length, DOWN)
        length += 1  
        
def leg(square, length, delta):
    "Complete one leg of given length, mutating `square` and yielding a copy at each step."
    for _ in range(length):
        square[:] = (X(square) + X(delta), Y(square) + Y(delta))
        yield tuple(square)  

print(cityblock_distance(nth(spiral(), N - 1)))

def spiralsums():
    "Yield the values of a spiral where each point has the sum of the 8 neighbors."
    value = defaultdict(int)
    for p in spiral():
        value[p] = sum(value[q] for q in neighbors8(p)) or 1
        yield value[p]

print(first(x for x in spiralsums() if x > N))

475
279138


In [11]:
## Advent Day 04

def isvalid(line):
    words = line.split()
    return len(words) == len(set(words))

print(quantify(Input(4), isvalid))

def isvalid2(line):
    words = mapt(canon, line.split())
    return len(words) == len(set(words))

print(quantify(Input(4), isvalid2))



477
167


In [12]:
## Advent Day 05

def jumps(): return [int(x) for x in Input(5).readlines()]

j = jumps()

def run(M):
    pc = 0
    for steps in count_from(1):
        oldpc = pc
        pc += M[pc]
        M[oldpc] += 1
        if pc not in range(len(M)):
            return steps
        
print(run(j))

j = jumps()

def run2(M, verbose=False):
    pc = 0
    for steps in count_from(1):
        oldpc = pc
        pc += M[pc]
        M[oldpc] += (-1 if M[oldpc] >= 3 else 1)
        if verbose: print(steps, pc, M)
        if pc not in range(len(M)):
            return steps
        
print(run2(j))

358309
28178177


In [13]:
## Advent Day 06

banks = vector(Input(6).read())

def realloc(banks):
    "How many cycles until we reach a configuration we've seen before?"
    seen = {banks}
    for cycles in count_from(1):
        banks = spread(banks)
        if banks in seen:
            return cycles
        seen.add(banks)
        
def spread(banks):
    "Find the area with the most blocks, and spread them evenly to following areas."
    banks  = list(banks)
    maxi   = max(range(len(banks)), key=lambda i: banks[i])
    blocks = banks[maxi]
    banks[maxi] = 0
    for i in range(maxi + 1, maxi + 1 + blocks):
        banks[i % len(banks)] += 1
    return tuple(banks)

print(realloc(banks))

banks = vector(Input(6).read())

def realloc2(banks):
    "When we hit a cycle, what is the length of the cycle?"
    seen = {banks: 0}
    for cycles in count_from(1):
        banks = spread(banks)
        if banks in seen:
            return cycles - seen[banks]
        seen[banks] = cycles

print(realloc2(banks))



5042
1086


In [14]:
## Advent Day 07

def towers(lines):
    "Return (weight, above) dicts."
    weight = {}
    above = {}
    for line in lines:
        name, w, *rest = re.findall(r'\w+', line)
        weight[name] = int(w)
        above[name] = rest
    return weight, above

weight, above = towers(Input(7))

programs = set(above)

print(programs - set(flatten(above.values())))

def wrong(p): return tower_weight(p) not in map(tower_weight, siblings(p))

def tower_weight(p): return weight[p] + sum(map(tower_weight, above[p]))

def siblings(p): 
    "The other programs at the same level as this one."
    return [] if p not in below else [s for s in above[below[p]] if s != p]

below = {a: b for b in programs for a in above[b]}

print(set(filter(wrong, programs)))

def wrongest(programs):
    return first(p for p in programs
                 if wrong(p) 
                 and not any(wrong(p2) for p2 in above[p]))

print(wrongest(programs))

def correct(p):
    "Return the weight that would make p's tower's weight the same as its sibling towers."
    delta = tower_weight(first(siblings(p))) - tower_weight(p) 
    return weight[p] + delta

print(correct(wrongest(programs)))

{'azqje'}
{'inwmb', 'nzeqmqi', 'rfkvap', 'azqje'}
rfkvap
646


In [22]:
## Advent Day 08

program = array(Input(8))

def run8(program):
    "Run the program and return final value of registers."
    registers = defaultdict(int)
    for (r, inc, delta, _if, r2, cmp, amount) in program:
        if operations[cmp](registers[r2], amount):
            registers[r] += delta * (+1 if inc == 'inc' else -1)
    return registers

print(max(run8(program).values()))


def run82(program):
    registers = defaultdict(int)
    highest = 0
    for r, inc, delta, _if, r2, cmp, amount in program:
        if operations[cmp](registers[r2], amount):
            registers[r] += delta * (+1 if inc == 'inc' else -1)
            highest = max(highest, registers[r])
    return highest

print(run82(program))


3745
4644


In [23]:
## Advent Day 09

text1 = Input(9).read()              # Read text
text2 = re.sub(r'!.', '', text1)     # Delete canceled characters
text3 = re.sub(r'<.*?>', '', text2)  # Delete garbage
text3

def total_score(text):
    "Total of group scores; each group scores one more than the group it is nested in."
    total = 0
    outer = [0] # Stack of scores of groups nested outside current group
    for c in text:
        if c == '{':
            score = outer[-1] + 1
            total += score
            outer.append(score)
        elif c == '}':
            outer.pop()
    return total

print(total_score(text3))

text4 = re.sub(r'<.*?>', '<>', text2)  # Delete inner garbage

print(len(text2) - len(text4))


14421
6817


In [67]:
## Advent Day 10

hashstring = list(range(0, 256))
lengths = [int(x) for x in Input(10).read().split(",")]
position = 0
skip = 0
hashlength = len(hashstring)


def strslice(st, start, end):
    if start == end:
        return ""
    return st[start:end]

def roundHash(hashstring, lengths, position, skip):
    for item in lengths:
        ##print(hashstring, item, position, skip)
        hashcycle = 2 * hashstring 
        sl = hashcycle[position:position+item]
        sl = sl[::-1]
        hashcycle = hashcycle[0:position] + sl + hashcycle[position+item:]
        #print(hashcycle)
        if position + item > hashlength:
            beg = position + item - hashlength
            hashcycle[0:beg] = hashcycle[hashlength:hashlength+beg]
        position = (position + item + skip) % hashlength
        skip = skip + 1
        hashstring = hashcycle[0:hashlength]
    return (hashstring, lengths, position, skip)
    
x = roundHash(hashstring, lengths, position, skip)
#print(x)
#print(x[0][0] * x[0][1])

lengths = [ord(x) for x in Input(10).read()] + [17, 31, 73, 47, 23]
#lengths = [ord(x) for x in "1,2,4"] + [17, 31, 73, 47, 23]
hashstring = list(range(0, 256))
position = 0
skip = 0
hashlength = len(hashstring)
for q in range(0, 64):
    hashstring, lengths, position, skip = roundHash(hashstring, lengths, position, skip)
    
#print(hashstring, lengths, position, skip)
hexa = []
for t in range(0, 256, 16):
    y = hashstring[t] ^ hashstring[t+1]
    for i in hashstring[t+2:t+16]:
        y = y ^ i
    hexa.append(y)
    
#print(hexa)
print( "".join(['{0:02x}'.format(a) for a in hexa]))

[101, 9, 93, 227, 76, 37, 200, 199, 111, 24, 254, 153, 52, 56, 212, 40, 34, 28, 64, 255, 44, 165, 12, 209, 152, 162, 225, 150, 238, 6, 171, 198, 168, 86, 220, 21, 83, 215, 133, 1, 173, 114, 62, 59, 117, 53, 161, 207, 110, 102, 237, 193, 77, 249, 253, 191, 61, 176, 95, 119, 116, 213, 94, 144, 159, 211, 115, 224, 160, 230, 58, 85, 126, 50, 164, 124, 169, 142, 66, 8, 43, 154, 221, 54, 130, 14, 148, 67, 118, 18, 145, 241, 108, 4, 22, 175, 3, 105, 172, 31, 87, 174, 185, 189, 73, 250, 90, 134, 17, 7, 38, 204, 218, 178, 23, 245, 166, 29, 80, 190, 46, 125, 239, 35, 141, 36, 112, 177, 55, 158, 242, 45, 247, 123, 51, 246, 20, 81, 2, 82, 15, 248, 79, 19, 197, 139, 68, 26, 206, 10, 205, 231, 226, 222, 30, 120, 240, 91, 70, 170, 25, 72, 57, 106, 33, 104, 92, 49, 208, 41, 188, 156, 99, 203, 138, 65, 100, 128, 11, 135, 233, 78, 219, 48, 179, 129, 184, 88, 232, 122, 214, 75, 39, 136, 194, 140, 127, 27, 186, 96, 195, 234, 132, 252, 103, 182, 121, 131, 89, 223, 109, 236, 137, 84, 151, 210, 107, 235, 243

105
